In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# DONNEES CARACTERISTIQUES

In [99]:
#Import de fichiers
annees1 = [2018]
dataframes1=[]
for annee in annees1:
    nom_fichier=f"caracteristiques_{annee}.csv"
    try:
        df=pd.read_csv(nom_fichier, encoding='latin1')
        dataframes1.append(df)
        print(f"Fichier {nom_fichier} importé avec succès.")
    except FileNotFoundError:
        print(f"Fichier {nom_fichier} introuvable")
    except Exception as e:
        print(f"Erreur lors de l'importation de {nom_fichier} : {e}")

annees2 = [2019,2020,2021,2023]
dataframes2=[]
for annee in annees2:
    nom_fichier=f"caracteristiques_{annee}.csv"
    try:
        df=pd.read_csv(nom_fichier, sep=';')
        dataframes2.append(df)
        print(f"Fichier {nom_fichier} importé avec succès.")
    except FileNotFoundError:
        print(f"Fichier {nom_fichier} introuvable")
    except Exception as e:
        print(f"Erreur lors de l'importation de {nom_fichier} : {e}")

df_combine1=pd.concat(dataframes1, ignore_index=True)
df_combine2=pd.concat(dataframes2, ignore_index=True)

#Annee 2022 - Renommage colonne Accident_Id' en 'Num_Acc'
caract22=pd.read_csv("caracteristiques_2022.csv", sep=';')
caract22 = caract22.rename(columns={'Accident_Id':'Num_Acc'})
caract22.head()

df_caract=pd.concat([df_combine1,df_combine2,caract22], ignore_index=True)
print(df_caract.info())
print(df_caract.head())
print(df_caract.isna().mean(axis=0)*100)

Fichier caracteristiques_2018.csv importé avec succès.
Fichier caracteristiques_2019.csv importé avec succès.
Fichier caracteristiques_2020.csv importé avec succès.
Fichier caracteristiques_2021.csv importé avec succès.
Fichier caracteristiques_2023.csv importé avec succès.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331009 entries, 0 to 331008
Data columns (total 16 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Num_Acc  331009 non-null  int64  
 1   an       331009 non-null  int64  
 2   mois     331009 non-null  int64  
 3   jour     331009 non-null  int64  
 4   hrmn     331009 non-null  object 
 5   lum      331009 non-null  int64  
 6   agg      331009 non-null  int64  
 7   int      331009 non-null  int64  
 8   atm      331004 non-null  float64
 9   col      331007 non-null  float64
 10  com      331009 non-null  object 
 11  adr      326475 non-null  object 
 12  gps      57733 non-null   object 
 13  lat      329186 non-null

In [101]:
#Supression des variables non pertinentes
df_caract_prep=df_caract.drop(['lat','long','adr','gps', 'com'], axis=1)

In [103]:
#Renommage des champs de la colonne année
df_caract_prep["an"] = df_caract_prep["an"].replace({18: 2018})

In [105]:
#Creation de tranches pour la colonne hrms
df_caract_prep["hrmn"]=df_caract_prep["hrmn"].astype(str)
df_caract_prep["hrmn"]=df_caract_prep["hrmn"].str.zfill(4)
df_caract_prep["heure"]=df_caract_prep["hrmn"].str[:2].astype(int)
df_caract_prep=df_caract_prep.drop(['hrmn'], axis=1)

In [107]:
#Creation de la colonne jour de semaine
df_caract_prep['date']=pd.to_datetime(df_caract_prep['an'].astype(str)+'-'+ df_caract_prep['mois'].astype(str)+'-'+df_caract_prep['jour'].astype(str))
df_caract_prep['jour_semaine']=df_caract_prep['date'].dt.day_name()
df_caract_prep["jour_semaine"] = df_caract_prep["jour_semaine"].replace({'Monday': 1, 'Tuesday': 2,'Wednesday': 3,'Thursday': 4,'Friday': 5,'Saturday': 6,'Sunday': 7}).astype(int)
df_caract_prep['jour_semaine'].head(8)
df_caract_prep=df_caract_prep.drop(['date', 'jour'], axis=1)

In [109]:
#Déterminer la proportion de valeurs -1 dans les variables qui en contiennent 
print(df_caract_prep["lum"].value_counts(normalize=True, dropna=False),end="\n\n")
print(df_caract_prep["int"].value_counts(normalize=True, dropna=False),end="\n\n")
print(df_caract_prep["atm"].value_counts(normalize=True, dropna=False),end="\n\n")
print(df_caract_prep["col"].value_counts(normalize=True, dropna=False),end="\n\n")

lum
 1    0.669257
 5    0.156316
 3    0.099082
 2    0.065766
 4    0.009553
-1    0.000027
Name: proportion, dtype: float64

int
 1    0.647372
 2    0.121695
 3    0.107227
 9    0.042954
 6    0.040229
 4    0.022033
 7    0.010976
 5    0.005658
 8    0.001816
-1    0.000042
Name: proportion, dtype: float64

atm
 1.0    0.794441
 2.0    0.109453
 8.0    0.037791
 3.0    0.022607
 7.0    0.018132
 5.0    0.006571
 9.0    0.004668
 4.0    0.003474
 6.0    0.002770
-1.0    0.000079
 NaN    0.000015
Name: proportion, dtype: float64

col
 6.0    0.303566
 3.0    0.297735
 2.0    0.130054
 7.0    0.098722
 1.0    0.098399
 4.0    0.036558
 5.0    0.030087
-1.0    0.004873
 NaN    0.000006
Name: proportion, dtype: float64



In [111]:
#Supression des valeurs -1
df_caract_prep=df_caract_prep[(df_caract_prep!=-1).all(axis=1)]
df_caract_prep.info()

<class 'pandas.core.frame.DataFrame'>
Index: 329356 entries, 0 to 331008
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Num_Acc       329356 non-null  int64  
 1   an            329356 non-null  int64  
 2   mois          329356 non-null  int64  
 3   lum           329356 non-null  int64  
 4   agg           329356 non-null  int64  
 5   int           329356 non-null  int64  
 6   atm           329351 non-null  float64
 7   col           329354 non-null  float64
 8   dep           329356 non-null  object 
 9   heure         329356 non-null  int32  
 10  jour_semaine  329356 non-null  int32  
dtypes: float64(2), int32(2), int64(6), object(1)
memory usage: 27.6+ MB


In [113]:
#Supression des valeurs nulles
print(df_caract_prep.isna().mean(axis=0)*100)
df_caract_prep=df_caract_prep.dropna()
print(df_caract_prep.info())

Num_Acc         0.000000
an              0.000000
mois            0.000000
lum             0.000000
agg             0.000000
int             0.000000
atm             0.001518
col             0.000607
dep             0.000000
heure           0.000000
jour_semaine    0.000000
dtype: float64
<class 'pandas.core.frame.DataFrame'>
Index: 329349 entries, 0 to 331008
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Num_Acc       329349 non-null  int64  
 1   an            329349 non-null  int64  
 2   mois          329349 non-null  int64  
 3   lum           329349 non-null  int64  
 4   agg           329349 non-null  int64  
 5   int           329349 non-null  int64  
 6   atm           329349 non-null  float64
 7   col           329349 non-null  float64
 8   dep           329349 non-null  object 
 9   heure         329349 non-null  int32  
 10  jour_semaine  329349 non-null  int32  
dtypes: float64(2), int32(2), in

# DONNEES LIEUX

In [115]:
#Import de fichiers
import warnings
warnings.filterwarnings('ignore')
annees = [2018]
dataframes=[]
for annee in annees:
    nom_fichier=f"lieux_{annee}.csv"
    try:
        df=pd.read_csv(nom_fichier, sep=',')
        dataframes.append(df)
        print(f"Fichier {nom_fichier} importé avec succès.")
    except FileNotFoundError:
        print(f"Fichier {nom_fichier} introuvable")
    except Exception as e:
        print(f"Erreur lors de l'importation de {nom_fichier} : {e}")
df_lieux1=pd.concat(dataframes, ignore_index=True)

annees = range(2019, 2024)
dataframes=[]
for annee in annees:
    nom_fichier=f"lieux_{annee}.csv"
    try:
        df=pd.read_csv(nom_fichier, sep=';')
        dataframes.append(df)
        print(f"Fichier {nom_fichier} importé avec succès.")
    except FileNotFoundError:
        print(f"Fichier {nom_fichier} introuvable")
    except Exception as e:
        print(f"Erreur lors de l'importation de {nom_fichier} : {e}")
df_lieux2=pd.concat(dataframes, ignore_index=True)

df_lieux=pd.concat([df_lieux1,df_lieux2], ignore_index=True)

Fichier lieux_2018.csv importé avec succès.
Fichier lieux_2019.csv importé avec succès.
Fichier lieux_2020.csv importé avec succès.
Fichier lieux_2021.csv importé avec succès.
Fichier lieux_2022.csv importé avec succès.
Fichier lieux_2023.csv importé avec succès.


In [117]:
#Exploration & pourcentage de valeurs manquantes
print(df_lieux.head())
print(df_lieux.info())
print(df_lieux.describe())
print("\n")
print(df_lieux.isna().mean()*100)

        Num_Acc  catr voie  v1   v2  circ  nbv   pr  pr1  vosp  prof  plan  \
0  201800000001     3   41 NaN    C   2.0  2.0  NaN  NaN   0.0   1.0   3.0   
1  201800000002     4   41 NaN    D   2.0  2.0  NaN  NaN   0.0   1.0   2.0   
2  201800000003     3   39 NaN    D   2.0  2.0  NaN  NaN   0.0   1.0   1.0   
3  201800000004     3   39 NaN  NaN   2.0  2.0  NaN  NaN   0.0   1.0   1.0   
4  201800000005     4  NaN NaN  NaN   1.0  1.0  NaN  NaN   0.0   1.0   1.0   

  lartpc larrout  surf  infra  situ  env1  vma  
0    NaN     NaN   1.0    0.0   1.0   0.0  NaN  
1    NaN     NaN   1.0    0.0   1.0   0.0  NaN  
2    NaN     NaN   1.0    0.0   1.0   0.0  NaN  
3    NaN     NaN   1.0    0.0   1.0   0.0  NaN  
4    NaN     NaN   1.0    0.0   1.0   0.0  NaN  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347047 entries, 0 to 347046
Data columns (total 19 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Num_Acc  347047 non-null  int64  
 1   catr

In [119]:
#Supression des variables non pertinentes
df_lieux_prep=df_lieux.drop(['voie','v1','v2','env1','lartpc','vma', 'pr','pr1','nbv', 'vosp', 'prof', 'plan','larrout'], axis=1)

In [121]:
#Pourcentage de valeurs manquantes
print(df_lieux_prep.info())
print(df_lieux_prep.isna().mean()*100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347047 entries, 0 to 347046
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Num_Acc  347047 non-null  int64  
 1   catr     347047 non-null  int64  
 2   circ     346642 non-null  float64
 3   surf     346605 non-null  float64
 4   infra    346581 non-null  float64
 5   situ     346553 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 15.9 MB
None
Num_Acc    0.000000
catr       0.000000
circ       0.116699
surf       0.127360
infra      0.134276
situ       0.142344
dtype: float64


In [123]:
# Pourcentage de valeurs renseignées à -1 par variable
print(df_lieux_prep["catr"].value_counts(normalize=True, dropna=False),end="\n\n")
print(df_lieux_prep["circ"].value_counts(normalize=True, dropna=False),end="\n\n")
print(df_lieux_prep["infra"].value_counts(normalize=True, dropna=False),end="\n\n")
print(df_lieux_prep["surf"].value_counts(normalize=True, dropna=False),end="\n\n")
print(df_lieux_prep["situ"].value_counts(normalize=True, dropna=False),end="\n\n")

catr
4    0.436045
3    0.368685
1    0.090772
2    0.063112
7    0.026299
6    0.006913
9    0.006890
5    0.001285
Name: proportion, dtype: float64

circ
 2.0    0.618544
 1.0    0.179379
 3.0    0.137111
-1.0    0.049575
 0.0    0.008005
 4.0    0.006218
 NaN    0.001167
Name: proportion, dtype: float64

infra
 0.0    0.841307
 5.0    0.058001
 9.0    0.029719
 2.0    0.016632
 3.0    0.013528
 1.0    0.011042
 6.0    0.009359
-1.0    0.009103
 8.0    0.006152
 4.0    0.003250
 NaN    0.001343
 7.0    0.000565
Name: proportion, dtype: float64

surf
 1.0    0.800681
 2.0    0.181344
 9.0    0.005345
 7.0    0.003449
 8.0    0.001651
 5.0    0.001640
 3.0    0.001596
 NaN    0.001274
 0.0    0.001032
-1.0    0.000784
 6.0    0.000712
 4.0    0.000493
Name: proportion, dtype: float64

situ
 1.0    0.788058
 3.0    0.123289
 8.0    0.026662
 5.0    0.019545
 4.0    0.018796
 6.0    0.009840
 2.0    0.009252
 0.0    0.002155
 NaN    0.001423
-1.0    0.000980
Name: proportion, dtype: floa

In [125]:
#Suppression les lignes renseignées à -1
df_lieux_prep=df_lieux_prep[(df_lieux_prep!=-1).all(axis=1)]
df_lieux_prep.info()
print(df_lieux_prep.isna().mean()*100)

<class 'pandas.core.frame.DataFrame'>
Index: 326682 entries, 0 to 347046
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Num_Acc  326682 non-null  int64  
 1   catr     326682 non-null  int64  
 2   circ     326277 non-null  float64
 3   surf     326240 non-null  float64
 4   infra    326216 non-null  float64
 5   situ     326188 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 17.4 MB
Num_Acc    0.000000
catr       0.000000
circ       0.123974
surf       0.135300
infra      0.142646
situ       0.151217
dtype: float64


In [127]:
# Supression des valeurs manquantes
print(df_lieux_prep.isna().mean()*100)
df_lieux_prep=df_lieux_prep.dropna()
print(df_lieux_prep.info())
print(df_lieux_prep.isna().mean()*100)

Num_Acc    0.000000
catr       0.000000
circ       0.123974
surf       0.135300
infra      0.142646
situ       0.151217
dtype: float64
<class 'pandas.core.frame.DataFrame'>
Index: 326028 entries, 0 to 347046
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Num_Acc  326028 non-null  int64  
 1   catr     326028 non-null  int64  
 2   circ     326028 non-null  float64
 3   surf     326028 non-null  float64
 4   infra    326028 non-null  float64
 5   situ     326028 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 17.4 MB
None
Num_Acc    0.0
catr       0.0
circ       0.0
surf       0.0
infra      0.0
situ       0.0
dtype: float64


# DONNEES USAGERS

In [129]:
#Import de fichiers
annees = [2018]
dataframes=[]
for annee in annees:
    nom_fichier=f"usagers_{annee}.csv"
    try:
        df=pd.read_csv(nom_fichier, sep=',')
        dataframes.append(df)
        print(f"Fichier {nom_fichier} importé avec succès.")
    except FileNotFoundError:
        print(f"Fichier {nom_fichier} introuvable")
    except Exception as e:
        print(f"Erreur lors de l'importation de {nom_fichier} : {e}")
df_usagers1=pd.concat(dataframes, ignore_index=True)

annees = range(2019, 2024)
dataframes=[]
for annee in annees:
    nom_fichier=f"usagers_{annee}.csv"
    try:
        df=pd.read_csv(nom_fichier, sep=';')
        dataframes.append(df)
        print(f"Fichier {nom_fichier} importé avec succès.")
    except FileNotFoundError:
        print(f"Fichier {nom_fichier} introuvable")
    except Exception as e:
        print(f"Erreur lors de l'importation de {nom_fichier} : {e}")
df_usagers2=pd.concat(dataframes, ignore_index=True)

df_usagers=pd.concat([df_usagers1,df_usagers2], ignore_index=True)
print(df_usagers.info())
print(df_usagers.describe())
print(df_usagers.head())

Fichier usagers_2018.csv importé avec succès.
Fichier usagers_2019.csv importé avec succès.
Fichier usagers_2020.csv importé avec succès.
Fichier usagers_2021.csv importé avec succès.
Fichier usagers_2022.csv importé avec succès.
Fichier usagers_2023.csv importé avec succès.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750140 entries, 0 to 750139
Data columns (total 17 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Num_Acc      750140 non-null  int64  
 1   place        739180 non-null  float64
 2   catu         750140 non-null  int64  
 3   grav         750140 non-null  int64  
 4   sexe         750140 non-null  int64  
 5   trajet       750026 non-null  float64
 6   secu         126040 non-null  float64
 7   locp         695489 non-null  float64
 8   actp         695494 non-null  object 
 9   etatp        695494 non-null  float64
 10  an_nais      741571 non-null  float64
 11  num_veh      750140 non-null  object 
 12  id_veh

In [131]:
#Pourcentage de valeurs manquantes
print(df_usagers.isna().mean()*100)
print("La variable secu prédente plus de 80% de valeurs manquantes, aussi elle peut être supprimée")
df_usagers_prep=df_usagers.drop(['secu'], axis=1)
print(df_usagers_prep.info())
print(df_usagers_prep.isna().mean()*100)

Num_Acc         0.000000
place           1.461061
catu            0.000000
grav            0.000000
sexe            0.000000
trajet          0.015197
secu           83.197803
locp            7.285440
actp            7.284774
etatp           7.284774
an_nais         1.142320
num_veh         0.000000
id_vehicule    17.352628
secu1          17.352628
secu2          17.352628
secu3          17.352628
id_usager      49.116298
dtype: float64
La variable secu prédente plus de 80% de valeurs manquantes, aussi elle peut être supprimée
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750140 entries, 0 to 750139
Data columns (total 16 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Num_Acc      750140 non-null  int64  
 1   place        739180 non-null  float64
 2   catu         750140 non-null  int64  
 3   grav         750140 non-null  int64  
 4   sexe         750140 non-null  int64  
 5   trajet       750026 non-null  float64
 6   locp    

In [133]:
#Définition de la fonction categorie_secu pour combiner les catégories des variables 'secu1','secu2','secu3' dans le df df_usagers
def categorie_secu(row):
    # Les trois variables sont dans {1, 2, 3, 4, 5, 6, 7} 'Trois équipements'
    if all(val in [1, 2, 3, 4, 5, 6, 7] for val in [row['secu1'], row['secu2'], row['secu3']]):
        return 3
    # Les trois variables sont dans {-1, 8, 9} : 'Non renseigné'
    elif all(val in [-1, 8, 9] for val in [row['secu1'], row['secu2'], row['secu3']]):
        return 4
    # Les trois variables sont à 0 :'Zéro équipement' 
    elif row['secu1'] == 0 and row['secu2'] == 0 and row['secu3'] == 0:
        return 0
    # Si au moins une des variables est dans {1, 2, 3, 4, 5, 6, 7}, alors : 'Au moins un équipement'
    elif any(val in [1, 2, 3, 4, 5, 6, 7] for val in [row['secu1'], row['secu2'], row['secu3']]):
        return 1
    # Autre
    else:
        return 2

In [135]:
#Remplacement des valeurs manquantes par -1 et application de la fonction categorie_secu
df_usagers_prep[['secu1','secu2','secu3']]=df_usagers_prep[['secu1','secu2','secu3']].fillna(-1)
df_usagers_prep['secu_combined'] = df_usagers_prep.apply(categorie_secu, axis=1)
print(df_usagers_prep['secu_combined'].value_counts(normalize=True, dropna=False)*100)

secu_combined
1    64.551017
4    22.361826
2    12.873730
0     0.120111
3     0.093316
Name: proportion, dtype: float64


In [137]:
#Suppression des colonnes 'secu1','secu2','secu3'
df_usagers_prep=df_usagers_prep.drop(['secu1','secu2','secu3'], axis=1)
print(df_usagers_prep.isna().mean()*100)
print(df_usagers_prep.info())

Num_Acc           0.000000
place             1.461061
catu              0.000000
grav              0.000000
sexe              0.000000
trajet            0.015197
locp              7.285440
actp              7.284774
etatp             7.284774
an_nais           1.142320
num_veh           0.000000
id_vehicule      17.352628
id_usager        49.116298
secu_combined     0.000000
dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750140 entries, 0 to 750139
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Num_Acc        750140 non-null  int64  
 1   place          739180 non-null  float64
 2   catu           750140 non-null  int64  
 3   grav           750140 non-null  int64  
 4   sexe           750140 non-null  int64  
 5   trajet         750026 non-null  float64
 6   locp           695489 non-null  float64
 7   actp           695494 non-null  object 
 8   etatp          695494 non-null  float64
 

In [139]:
#Analyse des variables 'actp','etatp', 'locp'
print(df_usagers_prep["catu"].value_counts())
print(df_usagers_prep["catu"].value_counts(normalize=True, dropna=False)*100)
print("Sur 741 903 usagers accidentés, 59 068 concernent des piétons, et 691 068 concernent le conducteur et les passagers")
print("Soit 8% de piétons et 92% de conducteur et passagers")
print("Nous pourrions décider de ne garder que les usagers hors piétons.")

catu
1    556713
2    134355
3     59072
Name: count, dtype: int64
catu
1    74.214547
2    17.910657
3     7.874797
Name: proportion, dtype: float64
Sur 741 903 usagers accidentés, 59 068 concernent des piétons, et 691 068 concernent le conducteur et les passagers
Soit 8% de piétons et 92% de conducteur et passagers
Nous pourrions décider de ne garder que les usagers hors piétons.


In [141]:
#Suppression des usagers de type piéton et des colonnes 'actp','etatp', 'locp'
df_usagers_prep=df_usagers_prep.loc[df_usagers_prep["catu"]!=3]
df_usagers_prep=df_usagers_prep.drop(['actp','etatp', 'locp'], axis=1)
print(df_usagers_prep.info())
print(df_usagers_prep.isna().mean()*100)

<class 'pandas.core.frame.DataFrame'>
Index: 691068 entries, 0 to 750139
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Num_Acc        691068 non-null  int64  
 1   place          691068 non-null  float64
 2   catu           691068 non-null  int64  
 3   grav           691068 non-null  int64  
 4   sexe           691068 non-null  int64  
 5   trajet         690960 non-null  float64
 6   an_nais        682506 non-null  float64
 7   num_veh        691068 non-null  object 
 8   id_vehicule    571859 non-null  object 
 9   id_usager      353095 non-null  object 
 10  secu_combined  691068 non-null  int64  
dtypes: float64(3), int64(5), object(3)
memory usage: 63.3+ MB
None
Num_Acc           0.000000
place             0.000000
catu              0.000000
grav              0.000000
sexe              0.000000
trajet            0.015628
an_nais           1.238952
num_veh           0.000000
id_vehicule      17.24996

In [143]:
#Pourcentage des valeurs renseignées à -1
print(df_usagers_prep["trajet"].value_counts(normalize=True, dropna=False)*100,end="\n\n")
print(df_usagers_prep["sexe"].value_counts(normalize=True, dropna=False)*100,end="\n\n")
print(df_usagers_prep["place"].value_counts(normalize=True, dropna=False)*100,end="\n\n")
print(df_usagers_prep["grav"].value_counts(normalize=True, dropna=False)*100,end="\n\n")

trajet
 5.0    37.542181
 0.0    24.836340
 1.0    13.946240
 4.0     9.914799
 9.0     8.027285
 3.0     2.719125
 2.0     1.658303
-1.0     1.340100
 NaN     0.015628
Name: proportion, dtype: float64

sexe
 1    68.950089
 2    29.858567
-1     1.191344
Name: proportion, dtype: float64

place
 1.0     80.436657
 2.0     12.227451
 3.0      2.074904
 4.0      1.862335
 9.0      1.135778
 7.0      1.039695
 5.0      0.579972
 8.0      0.491124
 6.0      0.148032
-1.0      0.003907
 10.0     0.000145
Name: proportion, dtype: float64

grav
 1    45.336783
 4    37.598760
 3    14.558915
 2     2.445345
-1     0.060197
Name: proportion, dtype: float64



In [145]:
#Supression des valeurs renseignées à -1
df_usagers_prep=df_usagers_prep[(df_usagers_prep['trajet']!=-1) & (df_usagers_prep['sexe']!=-1) & (df_usagers_prep['place']!=-1)]
df_usagers_prep.info()
print(df_usagers_prep.isna().mean()*100)
df_usagers_prep.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 681616 entries, 0 to 750139
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Num_Acc        681616 non-null  int64  
 1   place          681616 non-null  float64
 2   catu           681616 non-null  int64  
 3   grav           681616 non-null  int64  
 4   sexe           681616 non-null  int64  
 5   trajet         681508 non-null  float64
 6   an_nais        681357 non-null  float64
 7   num_veh        681616 non-null  object 
 8   id_vehicule    562407 non-null  object 
 9   id_usager      344325 non-null  object 
 10  secu_combined  681616 non-null  int64  
dtypes: float64(3), int64(5), object(3)
memory usage: 62.4+ MB
Num_Acc           0.000000
place             0.000000
catu              0.000000
grav              0.000000
sexe              0.000000
trajet            0.015845
an_nais           0.037998
num_veh           0.000000
id_vehicule      17.489173
id_

,Num_Acc,place,catu,grav,sexe,trajet,an_nais,secu_combined
count,6.816160e+05,681616.000000,681616.000000,681616.000000,681616.000000,681508.000000,681357.000000,681616.000000
mean,2.020479e+11,1.443459,1.196831,2.461731,1.302434,3.295690,1982.307824,1.715586
std,1.729148e+08,1.293172,0.397604,1.379874,0.459313,2.704114,18.239079,1.216436
min,2.018000e+11,1.000000,1.000000,1.000000,1.000000,0.000000,1900.000000,0.000000
25%,2.019000e+11,1.000000,1.000000,1.000000,1.000000,0.000000,1970.000000,1.000000
50%,2.021000e+11,1.000000,1.000000,3.000000,1.000000,4.000000,1986.000000,1.000000
75%,2.022000e+11,1.000000,1.000000,4.000000,2.000000,5.000000,1997.000000,2.000000
max,2.023001e+11,9.000000,2.000000,4.000000,2.000000,9.000000,2023.000000,4.000000


In [147]:
#Pourcentage de valeurs manquantes
print(df_usagers_prep["trajet"].value_counts(normalize=True, dropna=False)*100,end="\n\n")
print(df_usagers_prep["sexe"].value_counts(normalize=True, dropna=False)*100,end="\n\n")
print(df_usagers_prep["place"].value_counts(normalize=True, dropna=False)*100,end="\n\n")
print(df_usagers_prep["grav"].value_counts(normalize=True, dropna=False)*100,end="\n\n")

trajet
5.0    38.062780
0.0    25.156246
1.0    14.139633
4.0    10.051407
9.0     8.136106
3.0     2.756684
2.0     1.681299
NaN     0.015845
Name: proportion, dtype: float64

sexe
1    69.756578
2    30.243422
Name: proportion, dtype: float64

place
1.0    80.197941
2.0    12.379258
3.0     2.098689
4.0     1.884052
9.0     1.151381
7.0     1.053966
5.0     0.586841
8.0     0.497788
6.0     0.150085
Name: proportion, dtype: float64

grav
1    44.709044
4    38.066595
3    14.748920
2     2.475441
Name: proportion, dtype: float64



In [149]:
# Supression des valeurs manquantes des colonnes trajet et an_nais
print(df_usagers_prep.isna().mean()*100)
df_usagers_prep=df_usagers_prep.dropna(subset=['trajet','an_nais'])
df_usagers_prep.info()
print(df_usagers_prep.isna().mean()*100)

Num_Acc           0.000000
place             0.000000
catu              0.000000
grav              0.000000
sexe              0.000000
trajet            0.015845
an_nais           0.037998
num_veh           0.000000
id_vehicule      17.489173
id_usager        49.484020
secu_combined     0.000000
dtype: float64
<class 'pandas.core.frame.DataFrame'>
Index: 681250 entries, 0 to 750139
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Num_Acc        681250 non-null  int64  
 1   place          681250 non-null  float64
 2   catu           681250 non-null  int64  
 3   grav           681250 non-null  int64  
 4   sexe           681250 non-null  int64  
 5   trajet         681250 non-null  float64
 6   an_nais        681250 non-null  float64
 7   num_veh        681250 non-null  object 
 8   id_vehicule    562178 non-null  object 
 9   id_usager      344096 non-null  object 
 10  secu_combined  681250 non-null  int64

In [151]:
df_usagers_prep.describe()

,Num_Acc,place,catu,grav,sexe,trajet,an_nais,secu_combined
count,6.812500e+05,681250.000000,681250.000000,681250.000000,681250.000000,681250.000000,681250.000000,681250.000000
mean,2.020479e+11,1.443530,1.196834,2.462212,1.302528,3.296305,1982.307655,1.714955
std,1.728844e+08,1.293353,0.397606,1.379834,0.459353,2.703763,18.238934,1.216129
min,2.018000e+11,1.000000,1.000000,1.000000,1.000000,0.000000,1900.000000,0.000000
25%,2.019000e+11,1.000000,1.000000,1.000000,1.000000,0.000000,1970.000000,1.000000
50%,2.021000e+11,1.000000,1.000000,3.000000,1.000000,4.000000,1986.000000,1.000000
75%,2.022000e+11,1.000000,1.000000,4.000000,2.000000,5.000000,1997.000000,2.000000
max,2.023001e+11,9.000000,2.000000,4.000000,2.000000,9.000000,2023.000000,4.000000


# DONNES VEHICULES

In [153]:
#Import de fichiers
annees = [2018]
dataframes=[]
for annee in annees:
    nom_fichier=f"vehicules_{annee}.csv"
    try:
        df=pd.read_csv(nom_fichier, sep=',')
        dataframes.append(df)
        print(f"Fichier {nom_fichier} importé avec succès.")
    except FileNotFoundError:
        print(f"Fichier {nom_fichier} introuvable")
    except Exception as e:
        print(f"Erreur lors de l'importation de {nom_fichier} : {e}")
df_veh1=pd.concat(dataframes, ignore_index=True)

annees = range(2019, 2024)
dataframes=[]
for annee in annees:
    nom_fichier=f"vehicules_{annee}.csv"
    try:
        df=pd.read_csv(nom_fichier, sep=';')
        dataframes.append(df)
        print(f"Fichier {nom_fichier} importé avec succès.")
    except FileNotFoundError:
        print(f"Fichier {nom_fichier} introuvable")
    except Exception as e:
        print(f"Erreur lors de l'importation de {nom_fichier} : {e}")
df_veh2=pd.concat(dataframes, ignore_index=True)

df_veh=pd.concat([df_veh1,df_veh2], ignore_index=True)
print(df_veh.info())
print(df_veh.head())

Fichier vehicules_2018.csv importé avec succès.
Fichier vehicules_2019.csv importé avec succès.
Fichier vehicules_2020.csv importé avec succès.
Fichier vehicules_2021.csv importé avec succès.
Fichier vehicules_2022.csv importé avec succès.
Fichier vehicules_2023.csv importé avec succès.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566045 entries, 0 to 566044
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Num_Acc      566045 non-null  int64  
 1   senc         565913 non-null  float64
 2   catv         566045 non-null  int64  
 3   occutc       102788 non-null  float64
 4   obs          565856 non-null  float64
 5   obsm         565910 non-null  float64
 6   choc         565912 non-null  float64
 7   manv         565913 non-null  float64
 8   num_veh      566045 non-null  object 
 9   id_vehicule  467169 non-null  object 
 10  motor        467169 non-null  float64
dtypes: float64(7), int64(2), object(2)
me

In [155]:
#Exploration des données
print(df_veh.isna().sum())
print(df_veh.isna().mean()*100)
print(df_veh.info())
print(df_veh.describe())

Num_Acc             0
senc              132
catv                0
occutc         463257
obs               189
obsm              135
choc              133
manv              132
num_veh             0
id_vehicule     98876
motor           98876
dtype: int64
Num_Acc         0.000000
senc            0.023320
catv            0.000000
occutc         81.841020
obs             0.033390
obsm            0.023850
choc            0.023496
manv            0.023320
num_veh         0.000000
id_vehicule    17.467869
motor          17.467869
dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566045 entries, 0 to 566044
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Num_Acc      566045 non-null  int64  
 1   senc         565913 non-null  float64
 2   catv         566045 non-null  int64  
 3   occutc       102788 non-null  float64
 4   obs          565856 non-null  float64
 5   obsm         565910 non-null  float64


In [157]:
#Analyse de la variable occutc
print("La variable occutc prédente plus de 80% de valeurs manquantes, aussi elle peut être supprimée")
df_veh_prep=df_veh.drop(['occutc'], axis=1)
print(df_veh_prep.isna().mean()*100)

La variable occutc prédente plus de 80% de valeurs manquantes, aussi elle peut être supprimée
Num_Acc         0.000000
senc            0.023320
catv            0.000000
obs             0.033390
obsm            0.023850
choc            0.023496
manv            0.023320
num_veh         0.000000
id_vehicule    17.467869
motor          17.467869
dtype: float64


In [159]:
# Supression des variables senc et motor car non pertinentes
df_veh_prep=df_veh_prep.drop(['senc'], axis=1)
df_veh_prep=df_veh_prep.drop(['motor'], axis=1)

In [161]:
#Creation de catégorie pour simplifier la variable 'catv'
mapping_dict = {
    # 1. Véhicules légers
    '03': 1,
    '07': 1,
    '10': 1,

    # 2. Deux-roues motorisés
    '02': 2,
    '30': 2,
    '31': 2,
    '32': 2,
    '33': 2,
    '34': 2,

    # 3. Trois-roues motorisés
    '41': 3,
    '42': 3,
    '43': 3,

    # 4. Quads et véhicules tout-terrain
    '35': 4,
    '36': 4,

    # 5. Transports en commun
    '37': 5,
    '38': 5,

    # 6. Poids lourds & véhicules industriels
    '13': 6,
    '15': 6,
    '16': 6,
    '17': 6,

    # 7. Transports ferroviaires
    '19': 7,
    '39': 7,
    '40': 7,

    # 8. Engins spéciaux & agricoles
    '20': 8,
    '21': 8,

    # 9. Véhicules non motorisés & nouveaux modes
    '01': 9,
    '50': 9,
    '60': 9,
    '80': 9,
    
    # 10. Autres & indéterminés
    '00': 10,
    '99': 10,
    
    # Pour les codes non utilisés car manquants
    '04': 10,
    '05': 10,
    '06': 10,
    '08': 10,
    '09': 10,
    '11': 10,
    '12': 10,
    '18': 10,
    
}

# On s'assure que les codes soient au format chaîne sur deux caractères
df_veh_prep['catv'] = df_veh_prep['catv'].astype(str).str.zfill(2)

# Création de la nouvelle variable 'catv_modifié' en mappant la colonne 'catv'
df_veh_prep['catv_modifié'] = df_veh_prep['catv'].map(mapping_dict)

In [163]:
df_veh_prep=df_veh_prep.drop(['catv'], axis=1)
print(df_veh_prep.isna().mean()*100)
print(df_veh_prep.info())

Num_Acc          0.000000
obs              0.033390
obsm             0.023850
choc             0.023496
manv             0.023320
num_veh          0.000000
id_vehicule     17.467869
catv_modifié     0.758950
dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566045 entries, 0 to 566044
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Num_Acc       566045 non-null  int64  
 1   obs           565856 non-null  float64
 2   obsm          565910 non-null  float64
 3   choc          565912 non-null  float64
 4   manv          565913 non-null  float64
 5   num_veh       566045 non-null  object 
 6   id_vehicule   467169 non-null  object 
 7   catv_modifié  561749 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 34.5+ MB
None


In [165]:
#Pourcentage de valeurs manquantes
print(df_veh_prep["obs"].value_counts(normalize=True, dropna=False)*100,end="\n\n")
print(df_veh_prep["obsm"].value_counts(normalize=True, dropna=False)*100,end="\n\n")
print(df_veh_prep["choc"].value_counts(normalize=True, dropna=False)*100,end="\n\n")
print(df_veh_prep["manv"].value_counts(normalize=True, dropna=False)*100,end="\n\n")

obs
 0.0     85.419004
 1.0      2.333737
 13.0     1.792614
 2.0      1.504651
 4.0      1.372506
 3.0      1.332050
 6.0      1.154855
 8.0      1.144785
 12.0     0.767430
 14.0     0.755417
 15.0     0.573099
 9.0      0.466394
 16.0     0.440248
 7.0      0.308103
 11.0     0.181788
 5.0      0.180021
 10.0     0.110062
 17.0     0.090806
-1.0      0.039043
 NaN      0.033390
Name: proportion, dtype: float64

obsm
 2.0    70.065454
 0.0    18.360378
 1.0     9.537934
 9.0     1.456598
 6.0     0.331422
 4.0     0.099992
 5.0     0.081089
-1.0     0.043283
 NaN     0.023850
Name: proportion, dtype: float64

choc
 1.0    36.035828
 3.0    14.804477
 2.0    12.194790
 4.0     9.929069
 8.0     7.001740
 0.0     6.195090
 7.0     5.942107
 6.0     3.573214
 5.0     2.852070
 9.0     1.403069
-1.0     0.045049
 NaN     0.023496
Name: proportion, dtype: float64

manv
 1.0     40.663728
 2.0     11.706313
 15.0     8.122322
 0.0      6.466447
 13.0     4.425797
 17.0     3.359804
 9.0   

In [167]:
#Supression des valeurs renseignées à -1
df_veh_prep=df_veh_prep[(df_veh_prep!=-1).all(axis=1)]
df_veh_prep.info()
print(df_veh_prep.isna().mean()*100)

<class 'pandas.core.frame.DataFrame'>
Index: 565554 entries, 0 to 566044
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Num_Acc       565554 non-null  int64  
 1   obs           565365 non-null  float64
 2   obsm          565419 non-null  float64
 3   choc          565421 non-null  float64
 4   manv          565422 non-null  float64
 5   num_veh       565554 non-null  object 
 6   id_vehicule   466678 non-null  object 
 7   catv_modifié  561263 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 38.8+ MB
Num_Acc          0.000000
obs              0.033419
obsm             0.023870
choc             0.023517
manv             0.023340
num_veh          0.000000
id_vehicule     17.483034
catv_modifié     0.758725
dtype: float64


In [169]:
#Supression des valeurs manquantes
print(df_veh_prep.isna().mean()*100)
df_veh_prep=df_veh_prep.dropna(subset=['obs', 'obsm','choc','manv','catv_modifié'])
df_veh_prep.info()
print(df_veh_prep.isna().mean()*100)

Num_Acc          0.000000
obs              0.033419
obsm             0.023870
choc             0.023517
manv             0.023340
num_veh          0.000000
id_vehicule     17.483034
catv_modifié     0.758725
dtype: float64
<class 'pandas.core.frame.DataFrame'>
Index: 561036 entries, 0 to 566044
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Num_Acc       561036 non-null  int64  
 1   obs           561036 non-null  float64
 2   obsm          561036 non-null  float64
 3   choc          561036 non-null  float64
 4   manv          561036 non-null  float64
 5   num_veh       561036 non-null  object 
 6   id_vehicule   463158 non-null  object 
 7   catv_modifié  561036 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 38.5+ MB
Num_Acc          0.000000
obs              0.000000
obsm             0.000000
choc             0.000000
manv             0.000000
num_veh          0.000000
id_vehicule  

# DONNEES REGROUPEES

In [171]:
#Concatenation des 4 df précédents
caract_lieux=df_caract_prep.merge(right=df_lieux_prep, on='Num_Acc', how='left')
veh_usagers=df_veh_prep.merge(right=df_usagers_prep, on=['Num_Acc','id_vehicule'], how='left')
accidents=caract_lieux.merge(right=veh_usagers, on='Num_Acc', how='left')

In [172]:
accidents.head()
accidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 835172 entries, 0 to 835171
Data columns (total 32 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Num_Acc        835172 non-null  int64  
 1   an             835172 non-null  int64  
 2   mois           835172 non-null  int64  
 3   lum            835172 non-null  int64  
 4   agg            835172 non-null  int64  
 5   int            835172 non-null  int64  
 6   atm            835172 non-null  float64
 7   col            835172 non-null  float64
 8   dep            835172 non-null  object 
 9   heure          835172 non-null  int32  
 10  jour_semaine   835172 non-null  int32  
 11  catr           798787 non-null  float64
 12  circ           798787 non-null  float64
 13  surf           798787 non-null  float64
 14  infra          798787 non-null  float64
 15  situ           798787 non-null  float64
 16  obs            834128 non-null  float64
 17  obsm           834128 non-nul

In [175]:
#Creation de la colonne age afin que ce soit plus interprétable   
accidents['age']=accidents['an']-accidents['an_nais']
accidents['age']
accidents=accidents.drop(['an_nais'], axis=1)

In [177]:
#Vérification si présence de doublons
accidents.duplicated().sum()

17596

In [179]:
#Suppression des doublons
accidents=accidents.drop_duplicates(keep='first')

In [180]:
#Suppression des colonnes ID
accidents=accidents.drop(['num_veh_x','id_vehicule','num_veh_y','id_usager'], axis=1)

In [183]:
accidents=accidents.set_index('Num_Acc')

In [185]:
accidents.info()

<class 'pandas.core.frame.DataFrame'>
Index: 817576 entries, 201800000001 to 202200055302
Data columns (total 27 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   an             817576 non-null  int64  
 1   mois           817576 non-null  int64  
 2   lum            817576 non-null  int64  
 3   agg            817576 non-null  int64  
 4   int            817576 non-null  int64  
 5   atm            817576 non-null  float64
 6   col            817576 non-null  float64
 7   dep            817576 non-null  object 
 8   heure          817576 non-null  int32  
 9   jour_semaine   817576 non-null  int32  
 10  catr           781199 non-null  float64
 11  circ           781199 non-null  float64
 12  surf           781199 non-null  float64
 13  infra          781199 non-null  float64
 14  situ           781199 non-null  float64
 15  obs            816543 non-null  float64
 16  obsm           816543 non-null  float64
 17  choc           81

In [187]:
#Supression des dernières valeurs manquantes
print(accidents.isna().sum())
print(accidents.isna().mean()*100)

an                   0
mois                 0
lum                  0
agg                  0
int                  0
atm                  0
col                  0
dep                  0
heure                0
jour_semaine         0
catr             36377
circ             36377
surf             36377
infra            36377
situ             36377
obs               1033
obsm              1033
choc              1033
manv              1033
catv_modifié      1033
place            16583
catu             16583
grav             16583
sexe             16583
trajet           16583
secu_combined    16583
age              16583
dtype: int64
an               0.000000
mois             0.000000
lum              0.000000
agg              0.000000
int              0.000000
atm              0.000000
col              0.000000
dep              0.000000
heure            0.000000
jour_semaine     0.000000
catr             4.449372
circ             4.449372
surf             4.449372
infra            4.449372
si

In [189]:
accidents2=accidents.dropna()
print(accidents2.info())
print(accidents2.isna().mean()*100)

<class 'pandas.core.frame.DataFrame'>
Index: 765944 entries, 201800000001 to 202200055302
Data columns (total 27 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   an             765944 non-null  int64  
 1   mois           765944 non-null  int64  
 2   lum            765944 non-null  int64  
 3   agg            765944 non-null  int64  
 4   int            765944 non-null  int64  
 5   atm            765944 non-null  float64
 6   col            765944 non-null  float64
 7   dep            765944 non-null  object 
 8   heure          765944 non-null  int32  
 9   jour_semaine   765944 non-null  int32  
 10  catr           765944 non-null  float64
 11  circ           765944 non-null  float64
 12  surf           765944 non-null  float64
 13  infra          765944 non-null  float64
 14  situ           765944 non-null  float64
 15  obs            765944 non-null  float64
 16  obsm           765944 non-null  float64
 17  choc           76

In [191]:
accidents2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 765944 entries, 201800000001 to 202200055302
Data columns (total 27 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   an             765944 non-null  int64  
 1   mois           765944 non-null  int64  
 2   lum            765944 non-null  int64  
 3   agg            765944 non-null  int64  
 4   int            765944 non-null  int64  
 5   atm            765944 non-null  float64
 6   col            765944 non-null  float64
 7   dep            765944 non-null  object 
 8   heure          765944 non-null  int32  
 9   jour_semaine   765944 non-null  int32  
 10  catr           765944 non-null  float64
 11  circ           765944 non-null  float64
 12  surf           765944 non-null  float64
 13  infra          765944 non-null  float64
 14  situ           765944 non-null  float64
 15  obs            765944 non-null  float64
 16  obsm           765944 non-null  float64
 17  choc           76

In [193]:
#Export fichier nettoyé 
accidents2.to_csv('C:/Users/S40066/Documents/ProjetDS_Accidents/accidents.csv')